## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
!pip install pymorphy2

import pandas
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 5.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=730751503ab509dff8dbb4dc3a084a67b4009a5b8a0c3296edcf204ad4fb87e3
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!gdown 1DsQsbRV5Sm0IWyM5ZdEdU-v5QUdGe_v2
data = pandas.read_csv('/content/women-clothing-accessories.csv', sep = '\t')

Downloading...
From: https://drive.google.com/uc?id=1DsQsbRV5Sm0IWyM5ZdEdU-v5QUdGe_v2
To: /content/women-clothing-accessories.csv
100% 21.8M/21.8M [00:01<00:00, 17.5MB/s]


In [ ]:
# Токенизация
data['review'] = data.apply(lambda row: word_tokenize(row['review']), axis=1)

data

,review,sentiment
0,"[качество, плохое, пошив, ужасный, (, горловин...",negative
1,"[Товар, отдали, другому, человеку, ,, я, не, п...",negative
2,"[Ужасная, синтетика, !, Тонкая, ,, ничего, общ...",negative
3,"[товар, не, пришел, ,, продавец, продлил, защи...",negative
4,"[Кофточка, голая, синтетика, ,, носить, не, во...",negative
...,...,...
89995,"[сделано, достаточно, хорошо, ., на, ткани, сд...",positive
89996,"[Накидка, шикарная, ., Спасибо, большое, провд...",positive
89997,"[спасибо, большое, ), продовца, рекомендую, .....",positive
89998,"[Очень, довольна, заказом, !, Меньше, месяца, ...",positive


In [ ]:
#Приведение слов к нижнему регистру
def removeCapitalizedLetters(word):
    if word.isdigit():
        return 
    return word.lower() 

data['review'] = data.apply(lambda row: [removeCapitalizedLetters(word) for word in row['review']], axis = 1)

data

,review,sentiment
0,"[качество, плохое, пошив, ужасный, (, горловин...",negative
1,"[товар, отдали, другому, человеку, ,, я, не, п...",negative
2,"[ужасная, синтетика, !, тонкая, ,, ничего, общ...",negative
3,"[товар, не, пришел, ,, продавец, продлил, защи...",negative
4,"[кофточка, голая, синтетика, ,, носить, не, во...",negative
...,...,...
89995,"[сделано, достаточно, хорошо, ., на, ткани, сд...",positive
89996,"[накидка, шикарная, ., спасибо, большое, провд...",positive
89997,"[спасибо, большое, ), продовца, рекомендую, .....",positive
89998,"[очень, довольна, заказом, !, меньше, месяца, ...",positive


In [ ]:
# Удаление стоп слов
data['review'] = data.apply(lambda row: [word for word in row['review'] if word not in stopwords.words('russian')], axis = 1)

data

,review,sentiment
0,"[качество, плохое, пошив, ужасный, (, горловин...",negative
1,"[товар, отдали, другому, человеку, ,, получила...",negative
2,"[ужасная, синтетика, !, тонкая, ,, общего, пре...",negative
3,"[товар, пришел, ,, продавец, продлил, защиту, ...",negative
4,"[кофточка, голая, синтетика, ,, носить, возмож...",negative
...,...,...
89995,"[сделано, достаточно, ., ткани, сделан, рисуно...",positive
89996,"[накидка, шикарная, ., спасибо, большое, провд...",positive
89997,"[спасибо, большое, ), продовца, рекомендую, .....",positive
89998,"[очень, довольна, заказом, !, меньше, месяца, ...",positive


In [ ]:
# Лемматизация

def applyLemmatization(word):
    return analyzer.parse(word)[0].normal_form

analyzer = MorphAnalyzer()

data['review'] = data.apply(lambda row: [applyLemmatization(word) for word in row['review'] if word is not None and word.isalpha()], axis = 1)

data

,review,sentiment
0,"[качество, плохой, пошив, ужасный, горловина, ...",negative
1,"[товар, отдать, другой, человек, получить, пос...",negative
2,"[ужасный, синтетик, тонкий, общий, представить...",negative
3,"[товар, прийти, продавец, продлить, защита, мо...",negative
4,"[кофточка, голый, синтетик, носить, возможно]",negative
...,...,...
89995,"[сделать, достаточно, ткань, сделать, рисунок,...",positive
89996,"[накидка, шикарный, спасибо, большой, провдо, ...",positive
89997,"[спасибо, большой, продовца, рекомендовать, за...",positive
89998,"[очень, довольный, заказ, маленький, месяц, рб...",positive


In [ ]:
data['review'] = data.apply(lambda row: " ".join(row[0]), axis = 1)

data

,review,sentiment
0,качество плохой пошив ужасный горловина напере...,negative
1,товар отдать другой человек получить посылка л...,negative
2,ужасный синтетик тонкий общий представить карт...,negative
3,товар прийти продавец продлить защита мой согл...,negative
4,кофточка голый синтетик носить возможно,negative
...,...,...
89995,сделать достаточно ткань сделать рисунок замет...,positive
89996,накидка шикарный спасибо большой провдо линять...,positive
89997,спасибо большой продовца рекомендовать заказат...,positive
89998,очень довольный заказ маленький месяц рб курье...,positive


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import pandas

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataframe = pandas.DataFrame(columns = ['Vectorizer and range', 'precision', 'recall', 'f1-score'])

Xtrain, Xtest, Ytrain, Ytest = train_test_split(data['review'], data['sentiment'], test_size = 0.3)

In [ ]:
# Векторизация

for i in range(2, 7):
  for j in range(i, 7):
    countVectorizer = CountVectorizer(ngram_range=(i, j))
    vectorizedXtrain = countVectorizer.fit_transform(Xtrain)
    vectorizedXtest = countVectorizer.transform(Xtest)

    model = MultinomialNB()
    model.fit(vectorizedXtrain, Ytrain)

    predictions = model.predict(vectorizedXtest)
    report = classification_report(Ytest, predictions).split()

    dataframe.loc[len(dataframe.index)] = [f'word CountVectorizer range:({i}, {j})', report[-4], report[-3], report[-2]]

In [ ]:
for i in range(3, 8):
  for j in range(i + 1, 9):
    countVectorizer = CountVectorizer(analyzer='char',ngram_range=(i, j))
    vectorizedXtrain = countVectorizer.fit_transform(Xtrain)
    vectorizedXtest = countVectorizer.transform(Xtest)

    model = MultinomialNB()
    model.fit(vectorizedXtrain, Ytrain)

    predictions = model.predict(vectorizedXtest)
    report = classification_report(Ytest, predictions).split()

    dataframe.loc[len(dataframe.index)] = [f'char CountVectorizer range:({i}, {j})', report[-4], report[-3], report[-2]]

In [ ]:
for rangeStart in range(2, 4):
  for rangeEnd in range(rangeStart, 4):
    for maximumDf in [0.1, 0.5]:
      for minimumDf in [0, 0.01]:
        for maximumFeatures in [4000, 25000]:
          vectorizer = TfidfVectorizer(ngram_range = (rangeStart, rangeEnd), max_df = maximumDf, min_df = minimumDf, max_features = maximumFeatures)
          vectorizedXtrain = vectorizer.fit_transform(Xtrain)
          vectorizedXtest = vectorizer.transform(Xtest)

          model = MultinomialNB()
          model.fit(vectorizedXtrain, Ytrain)

          predictions = model.predict(vectorizedXtest)
          report = classification_report(Ytest, predictions).split()

          dataframe.loc[len(dataframe.index)] = [f'TfidfVectorizer range:({rangeStart};{rangeEnd}), maximumDf:{maximumDf}, minimumDf:{minimumDf}, maximumFeatures: {maximumFeatures}', report[-4], report[-3], report[-2]]

In [ ]:
dataframe = dataframe.sort_values(by = ["f1-score"], ascending = False).head(20)

dataframe.set_index('Vectorizer and range')

dataframe

,Vectorizer and range,precision,recall,f1-score
27,"char CountVectorizer range:(6, 7)",0.71,0.70,0.71
25,"char CountVectorizer range:(5, 7)",0.71,0.71,0.71
19,"char CountVectorizer range:(3, 8)",0.71,0.70,0.71
23,"char CountVectorizer range:(4, 8)",0.71,0.71,0.71
29,"char CountVectorizer range:(7, 8)",0.71,0.70,0.71
26,"char CountVectorizer range:(5, 8)",0.72,0.71,0.71
28,"char CountVectorizer range:(6, 8)",0.71,0.71,0.71
20,"char CountVectorizer range:(4, 5)",0.71,0.70,0.70
17,"char CountVectorizer range:(3, 6)",0.71,0.70,0.70
18,"char CountVectorizer range:(3, 7)",0.71,0.70,0.70


**Результаты:**


Лучшим оказался векторайзер символьные n-граммы. Причем лучшие результаты по совокупности параметров получены при следующих параметрах диапазона 6, 7.


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

abcx нет, потому что findall возвращает список всех **непересекающихся** совпадений. А в данном случае abca пересекается с abca

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(' ..|^..', 'aaaa bbb cc hello')
print(result)

['aa', ' bb', ' cc', ' he']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('\.', 'Aaaa, aaaa; aa . Bbbbb. Ccccc. Ddddd. Eeeeeee.', maxsplit=3) 
print(result)

['Aaaa, aaaa; aa ', ' Bbbbb', ' Ccccc', ' Ddddd. Eeeeeee.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('\d', 'DIG', '789 avg8 09 ashd7bb')
print (result)

DIGDIGDIG avgDIG DIGDIG ashdDIGbb


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('http://\S+|https://\S+', '', 'hello use these links please: https://colab.research.google.com/ , https://piazza.com/class/ and http://localhost:8080')
print (result)

hello use these links please:  ,  and 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[a-z|A-Z|а-я|А-Я|Ё|ё]{3,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё. А вот on столе lays dog")

['Слова',
 'больше',
 'ещё',
 'больше',
 'слов',
 'Что',
 'ещё',
 'вот',
 'столе',
 'lays',
 'dog']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile('@\S+')
prog.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com,', '@test.in,', '@analyticsvidhya.com,', '@rest.biz']